<img src="logo.png">

# Diagnósticos de regresión.

Para que el modelo de regresión lineal sea fiable, en las conclusiones derivadas de las estimaciones de intervalos e inferencias (intervalos de confianza, contrastes de hipótesis,...) que realizamos a partir de dicho modelo se tienen que verificar cietas hipótesis.

Las tareas que realizan dichas verificacones se denominan **diagnósticos de regresión**.

Estos se clasifican en tres categorías:

* Errores: los errores deben seguir una $N(0,\sigma)$ con la misma varianza y ser incorrelacionados.


* Modelo: los puntos se tienen que ajustar a la estructura lineal considerada.


* Observaciones anómalas: a veces unas cuántas observaciones no se ajustan al modelo y hay que detectarlas.


En general hay dos métodos: gráficos y numéricos. 

## Homocedasticidad.

Uno de los problemas que puede sufrir nuestro modelo es que la varianza de los residuos no sea constante. 

**Ejemplo.**

Instalemos el paquete **gridExtra**.

Tomemos la tabla **homocedasticidad.csv** de nuestro repositorio y al cargarla en R llamémosla ``datos``. Las siguientes instrucciones nos graficarán la tabla junto con la recta de regresión lineal por mínimos cuadrados y el gráfico conocido como "ajustes vs error" y nos presentarán ambos gráficos (uno junto al otro)

``plot1 <- ggplot(data = datos) +
  geom_point(mapping = aes(x = x, y = y)) +
  geom_abline(mapping = aes(intercept = lm(y~x,datos)$coefficients[1],slope = lm(y~x,datos)$coefficients[2]),col = "red") ``
  
``plot2 <- ggplot() +
  geom_point(mapping = aes(x=lm(y~x,datos)$fitted.values,y=lm(y~x,datos)$res)) +
  geom_abline(mapping = aes(intercept = 0, slope = 0), col = "red")``

``grid.arrange(plot1,plot2,ncol=2)``

**Si un modelo es homocedástico, entonces el gráfico "ajustes vs errores" debe tomar una forma de "cielo estrellado".**


### El Test de White

Para aplicar el Test de White, hay que seguir los siguientes pasos:

* Obtener los residuos $\{e_i\}_{i=1}^n$ de la regresión lineal.


* Calcular el coeficiente de determinación $R^2$ de la regresión lineal de los $e_i^2$ respecto de las variables iniciales, sus cuadrados y los productos cruzados dos a dos. Es decir, calcular el coeficiente de determinación del modelo en la regresión siguiente: $$\begin{array}{rcl}\mu_{E^2|_{x_1,...,x_k,x_1^2,....x_k^2,x_ix_j(i,j=1,...,n,i<j)}}&=&\beta_0+\beta_1x_1+...+\beta_kx_k+\\
&&\beta_1^{(2)}x_1^2+...+\beta_k^{(2)}x_k^2+\\
&&\beta_{12}x_1x_2+...+\beta_{k-1,k}x_{k-1,k}.
\end{array}$$


* Calcular el estadístico de contraste $X_0=nR^2$, el cual, suponiendo que la varianza es constante, sigue una $\chi^2_q$, donde $q$ es el número de variables independientes de la regresión del paso anterior.


* Calculamos el p-valor $P(\chi^2_q\ge X_0)$ con el significado usual.

Para usar el Test de White en R, usamos la función ``bptest`` del paquete **lmtest**:

``bptest(lm1,~ U + I(U^2))`` 

donde

**lm1**: es el objeto de R donde hemos guardado la información de la regresión original.

**U**: es la matriz que contiene los valores de la muestra de las variables independientes.

**Observación importante.**

Supongamos que tenemos $n$ observaciones. Del capítulo anterior, sabemos que para realizar una regresión lineal debemos tener $n>$número de variables independientes que participan en el modelo. En el modelo del test de White, es sencillo ver que hay $2k+\binom{k}{2}=\frac{k(k+3)}{2}$. Por lo tanto, para aplicar el test de White se requiere $$n>\frac{k(k+3)}{2}$$ 


### Test de Breusch-Pagan.

Cuando no es posible aplicar el Test de White debido a la **Observación importante** anterior, en su lugar podemos aplicar el Test de Breuch-Pagan. Su aplicación es bastante parecida al de White, pero evita los términos de segundo orden en la regresión auxiliar:

* Obtener los residuos $\{e_i\}_{i=1}^n$ de la regresión lineal inicial.


* Calcular el coeficiente de determinación $R^2$ de la regresión lineal de los $e_i^2$ respecto de las variables iniciales. Es decir, calcular el coeficiente de determinación del modelo siguiente: $$\mu_{E^2|_{x_1,...,x_k}}=\beta_0+\beta_1x_1+\cdots+\beta_kx_k$$


* Calcular el estadístico $X_0=nR^2$ el cual, suponiendo homocedasticidad, sigue una $\chi^2_k$.


* Calculamos el p-valor $P(\chi^2_k\ge X_0)$ con el significado usual.


Para realizarlo en R, simplemente usamos ``bptest(lm1)`` donde lm1 es el objeto de R donde hemos guardado la información de la regresión original.

## Normalidad de los residuos.

Para detectar la normalidad de los residuos, podemos aplicar todos los tests de normalidad vistos en el tema de Bondad de Ajuste:

* Test Kolmogorov-Smirnov: tomar $\mu=0$ y para estimar $\sigma$ usamos la desviación muestral.


* Test Kolmogorov-Smirnov-Lilliefors: cuando la muestra es pequeña.


* Test Anderson-Darling.


* Test de Shapiro-Wilks.


* Test D'Agostino-Pearson.


Es aconsejable también realizr las pruebas gráficas de normalidad (como los QQ-plots).

## Correlación de los residuos: Test de Durbin-Watson.

Otra de las hipótesis que se deben verificar para que el análisis de regresión sea correcto es la incorrelación de los residuos.

La autocorrelación de los residuos puede ser de dos tipos:

* Positiva: un valor positivo (negativo) de un error genera una cadena de residuos positivos (negativos)


* Negativa: los residuos alternan signo.


Para comprobar que los residuos no presentan correlación, se puede aplicar el Test de Durbin-Watson.

Sean $\{e_i\}_{i=1}^n$ los residuos de la regresión. Tomemos $E_i$ y $E_{i-1}$ como las variables aleatorias de error y consideremos la recta de regresión de $E_i$ respecto de $E_{i-1}$. Esto es, consideremos el modelo $E_i=\beta_0+\beta_1E_{i-1}$.

Se plantea el contraste $$\left\{\begin{array}{l}\mathcal{H}_0:\beta_1=0\\\mathcal{H}_1:\beta_1\neq0\end{array}\right.$$ con el estadístico de contraste $$d=\frac{\sum_{i=2}^n(e_i-e_{i-1})^2}{\sum_{i=1}^ne_i^2}$$

El valor de este estadístico es aproximadamente $2(1-b_1)$ donde $b_1$ es una estimación de $\beta_1$. Si $\mathcal{H}_0$ es cierta, su distribución es la de una cierta combinación lineal de $\chi^2$.

Este test está implementado en la función ``dwtest`` del paquete **lmtest**. Su sintaxis es

``dwtest(lm1,alternative=...)``

donde 

**lm1**: es la regresión original.

**alternative** sirve para indicar si se testea autocorrelación positiva o negativa ("greater" o "less").


## Aditividad y linealidad

Cuando se plantea un modelo lineal, se suponen implícitamente las condiciones siguientes:

* Aditividad: para cada variable independiente $X_i$, la variación de $\mu_{Y|_{x_1,...,x_k}}$ asociada con un aumento en $X_i$ (manteniendo las otras variables constantes) es la misma sean cuales sean los valores de las otras variables independientes. Es decir, 

$$
\begin{array}{rcl}
\mu_{Y|_{x_1,...,x_i+h,...x_k}}-\mu_{Y|_{x_1,...,x_i,...x_k}}&=&\beta_0+\beta_1x_1+...+\beta_i(x_i+h)+...+\beta_kx_k-(\beta_0+\beta_1x_1+...+\beta_ix_i+...+\beta_kx_k)\\
&=&\beta_ih
\end{array}
$$

Observemos que la variación en $\mu$ es independiente del resto de las variables. Por otro lado:

* Linealidad: para cada variable independiente $X_i$, la variación de $\mu_{Y|_{x_1,...,x_k}}$ asociada con un aumento en $X_i$ (manteniendo las otras variables constantes) es la misma sea cual sea el valor de $X_i$ (es decir, en la expresión de arriba no aparece $X_i$). 



### Aditividad.

Podemos comprobar la aditividad con el Test de Tukey usando los llamados gráficos de residuos parciales para la linealidad. La idea principal es verificar que no haya interacción entre las variables independienes y así, cada una tendrá un efecto aditivo en el modelo.

Si existe la interacción, algunos de los términos cuadráticos tendrán peso en el modelo. Esta es la base del Test de Tukey.

Para aplicarlo, se siguen los siguientes pasos:

* Se obtienen los valores ajustados $\{\widehat{y_i}\}$ por la regresión lineal inicial.


* Se lleva a cabo una segunda regresión lineal incluyendo como nueva variable independiente los $\widehat{y_i}^2$. Sea $\beta$ el coeficiente de esta nueva variable.


* Se testea si la variable $\widehat{y}^2$ es significativa en la segunda regresión. Es decir, se realiza el contraste $$\left\{\begin{array}{l}\mathcal{H}_0:\beta=0\\\mathcal{H}_1:\beta\neq0\end{array}\right..$$

Si no podemos descartar $\mathcal{H}_0$, la variable de los $\widehat{y_i}^2$ no es significativa y el modelo es aditivo.

En R, para realizar el Test de Tukey hay que usar la función ``residualPlots`` del paquete **car**:

``residualPlots(lm1)``

donde 

**lm1** es el objeto de R donde hemos guardado la información de la regresión original

**plot** es un parámetro lógico. Si vale TRUE, nos dibuja los gráficos de los residuos frente a las variables regresoras y frente a los valores estimados junto con una curva de color azul indicando su tendencia; si vale FALSE simplemente da el valor del estadístico de Tukey y su p-valor.


Si optamos por ver los gráficos, entonces para concluir satisfactoriamente todos ellos deben parecer cielos estrellados.



### Linealidad.

Los gráficos de residuos parciales son una herramienta útil para detectar la no linealidad en una regresión. Se definen los residuos parciales $e_{ij}$ para la variable independiente $X_j$ como $$e_{ij}=e_i+b_jx_{ij},$$ donde $e_i$ es el residuo $i$-ésimo de la regresión lineal; $b_j$ es el coeficiente de $X_j$ en la regresión original y $x_{ij}$ es la observación $j$-ésima del individuo $i$-ésimo.

Los residuos parciales se dibujan contra los valores de $x_j$ y se hace su recta de regresión.

Si esta no se ajusta a la curva dada por una regresión no paramétrica suave (las variables independientes no están predeterminadas y se construyen con los datos), el modelo no es lineal.

La función de $R$ para representar estos gráficos es ``crPlots`` del paquete **car**: ``crPlots(lm1)``.

## Observaciones anómalas.

Las observaciones anómalas pueden provocar que se malinterpreten patrones en el conjunto de datos. Además, puntos aislados pueden tener una gran influencia en el modelo de regresión, dando resultados completamente diferentes. Por ejemplo, pueden provocar que nuestro modelo no capture características importantes de los datos. 

Por ello, es importante detectarlas.

Existen tres tipos de observaciones anómalas:

* **Outliers de regresión** son observaciones que tienen un valor anómalo de la variable $Y$, condicionado a los valores de sus variables independientes $X_i$. Tendrán un residuo muy alto pero no pueden afectar demasiado a los coeficientes de la regresión.


* **Leverages.** son observaciones con un valor anómalo de las variables $X_i$. No tienen por qué afectar los coeficientes de la regresión.


* **Observaciones influyentes** son aquellas que tienen un leverage alto; son outliers de regresión y afectan fuertemente a la regresión.


### Leverages

Para hallar las observaciones que son leverages, necesitamos la matriz Hat: $H=X(X^TX)^{-1}X^T$. Como $\widehat{y}=Hy$, tenemos que $\widehat{y_j}=h_{1j}y_1+h_{2j}y_2+...+h_{nj}y_n=\sum_{i=1}^nh_{ij}x_i$ para $j=1,...,n$.

Si $h_{ij}$ es grande, la observación $i$-ésima tiene un impacto sustancial en el valor predicho $j$-ésimo.

Se define el leverage de la observación $i$ como su valor hat: $h_i=\sum_{j=1}^nh_{ij}^2$.
hat
La regla de decisión que nos dirá si una observación tiene leverage grande (y por lo tanto debe ser considerada con cuidado) es cuando su valor hat es mayor que $2(k+1)/n$.

La función ``hatvalues`` de R calcula los valores hat basándose en la regla anterior: ``which(hatvalues(lm1)>2(k+1)/n)``.

### Outliers

Para detectar outliers, en R se usa la ``función outlierTest`` del paquete **car**:  ``outlierTest(lm1)``. 

### Observaciones influyentes.

El método que usaremos es la llamada Distanca de Cook. Las observaciones influyentes son aquellas cuya distancia de Cook es mayor a $4/(n-k-1)$

En R, la distancia de Cook se calcula con ``cooks.distance`` del paquete **car**: ``which(cooks.distance(lm1)>4/(n-k-1))``

### Tratamiento de las observaciones anómalas.

El tratamiento suele ser complejo. 

Nos debemos preguntar si se deben a errores en la entrada o recogida de los datos y si este es el caso, eliminarlas.

Pero también pueden explicar que no se ha considerado alguna variable independiente que afecta al conjunto de observaciones anómalas.

Las más peligrosas son las influyentes.

En el supuesto de que se determine que se pueden eliminar, se tienen que eliminar de una a una actualizando el modelo cada vez. 

**Ejemplo.** Se probó un modelo de simulación para el flujo máximo de agua de las cuencas hidrográficas comparando el flujo máximo medido de 10 tormentas con predicciones del flujo máximo obtenido del modelo de simulación. Q_o y Q_p son los flujos máximos observados y pronosticados, respectivamente. Se registraron cuatro variables independientes:

    * X_1: area de la cuenca (m^2),

    * X_2: pendiente promedio de la cuenca (en porcentaje),

    * X_3: índice de absorbencia superficial (0 = absorbencia completa, 100 = sin absorbencia), y

    * X_4: intensidad de pico de lluvia calculada en intervalos de media hora.

|Qo | Qp| x1| x2|x3 |x4|
|:--:|:--:|:--:|:--:|:--:|:--:|
|28|              32|              .03|             3.0|             70|              .6|
|112  |           142     |        .03  |           3.0    |         80 |             1.8|
|398  |           502      |       .13  |           6.5    |         65 |             2.0|
|772  |           790   |          1.00   |         15.0  |          60   |           .4|
|2294  |          3075|            1.00  |          15.0|            65|              2.3|
|2484   |         3230  |          3.00    |        7.0     |        67 |             1.0|
|2586    |        3535   |         5.00 |           6.0 |            62|              .9|
|3024   |         4265 |           7.00    |        6.5     |        56      |        1.1|
|4179   |         6529   |         7.00    |        6.5     |        56|              1.4|
|710    |         935  |           7.00   |         6.5   |          56   |           .7|



Consideramos Y=ln(Q_o/Q_p) como variable dependiente, consideramos la regresión de Y como función de X_1, X_2, X_3 y X_4. Se pide:

a) Estimar los valores b_0, b_1, b_2, b_3, b_4 para la regresión lineal de Y en función de X_i, i=1,2,3,4.

b) Hallar un intervalo de confianza al 95% de confianza para los parámetros beta_i, i=0,1,2,3,4.

c) Calcular la estimación de la varianza común de los errores de la regresión sigma^2.

d) Hallar el coeficiente de regresión y el coeficiente de regresión ajustado.

e) Estudiar si el modelo es homocedástico gráficamente y usando el test correspondiente.

f) Estudiar la normalidad de los residuos.

g) Estudiar la correlación de los residuos.

h) Contrastar la linealidad y la aditividad del modelo.

i) Hallar las observaciones "outliers", los "leverages" y las observaciones influyentes.

